In [167]:
import pandas as pd
import numpy as np
df = pd.read_csv("C:/Users/ADMIN/Downloads/Data/healthcare_dataset.csv")
df

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Tiffany Ramirez,81,Female,O-,Diabetes,2022-11-17,Patrick Parker,Wallace-Hamilton,Medicare,37490.98,146,Elective,2022-12-01,Aspirin,Inconclusive
1,Ruben Burns,35,Male,O+,Asthma,2023-06-01,Diane Jackson,"Burke, Griffin and Cooper",UnitedHealthcare,47304.06,404,Emergency,2023-06-15,Lipitor,Normal
2,Chad Byrd,61,Male,B-,Obesity,2019-01-09,Paul Baker,Walton LLC,Medicare,36874.90,292,Emergency,2019-02-08,Lipitor,Normal
3,Antonio Frederick,49,Male,B-,Asthma,2020-05-02,Brian Chandler,Garcia Ltd,Medicare,23303.32,480,Urgent,2020-05-03,Penicillin,Abnormal
4,Mrs. Brandy Flowers,51,Male,O-,Arthritis,2021-07-09,Dustin Griffin,"Jones, Brown and Murray",UnitedHealthcare,18086.34,477,Urgent,2021-08-02,Paracetamol,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,James Hood,83,Male,A+,Obesity,2022-07-29,Samuel Moody,"Wood, Martin and Simmons",UnitedHealthcare,39606.84,110,Elective,2022-08-02,Ibuprofen,Abnormal
9996,Stephanie Evans,47,Female,AB+,Arthritis,2022-01-06,Christopher Yates,Nash-Krueger,Blue Cross,5995.72,244,Emergency,2022-01-29,Ibuprofen,Normal
9997,Christopher Martinez,54,Male,B-,Arthritis,2022-07-01,Robert Nicholson,Larson and Sons,Blue Cross,49559.20,312,Elective,2022-07-15,Ibuprofen,Normal
9998,Amanda Duke,84,Male,A+,Arthritis,2020-02-06,Jamie Lewis,Wilson-Lyons,UnitedHealthcare,25236.34,420,Urgent,2020-02-26,Penicillin,Normal


# Bài toán:dự đoán kết quả khám bệnh (Test Result)

## Chọn ra những cột dữ liệu cần thiết để dự đoán

In [168]:
df = df[["Gender","Blood Type","Medical Condition","Admission Type","Medication","Insurance Provider","Test Results"]]
df

,Gender,Blood Type,Medical Condition,Admission Type,Medication,Insurance Provider,Test Results
0,Female,O-,Diabetes,Elective,Aspirin,Medicare,Inconclusive
1,Male,O+,Asthma,Emergency,Lipitor,UnitedHealthcare,Normal
2,Male,B-,Obesity,Emergency,Lipitor,Medicare,Normal
3,Male,B-,Asthma,Urgent,Penicillin,Medicare,Abnormal
4,Male,O-,Arthritis,Urgent,Paracetamol,UnitedHealthcare,Normal
...,...,...,...,...,...,...,...
9995,Male,A+,Obesity,Elective,Ibuprofen,UnitedHealthcare,Abnormal
9996,Female,AB+,Arthritis,Emergency,Ibuprofen,Blue Cross,Normal
9997,Male,B-,Arthritis,Elective,Ibuprofen,Blue Cross,Normal
9998,Male,A+,Arthritis,Urgent,Penicillin,UnitedHealthcare,Normal


## Kiểm tra các loại dữ liệu khác nhau của từng cột

In [169]:
for column in df.columns:
    if not pd.api.types.is_numeric_dtype(df[column]):
        unique_values = df[column].unique()
        print(f'{column}: {unique_values}')

Gender: ['Female' 'Male']
Blood Type: ['O-' 'O+' 'B-' 'AB+' 'A+' 'AB-' 'A-' 'B+']
Medical Condition: ['Diabetes' 'Asthma' 'Obesity' 'Arthritis' 'Hypertension' 'Cancer']
Admission Type: ['Elective' 'Emergency' 'Urgent']
Medication: ['Aspirin' 'Lipitor' 'Penicillin' 'Paracetamol' 'Ibuprofen']
Insurance Provider: ['Medicare' 'UnitedHealthcare' 'Aetna' 'Cigna' 'Blue Cross']
Test Results: ['Inconclusive' 'Normal' 'Abnormal']


## Mã hóa các cột độc lập

In [170]:
# Chọn các cột cần mã hóa
columns_to_encode = ['Gender',"Blood Type", 'Medical Condition', 'Admission Type', 'Medication',"Insurance Provider"]

# Mã hóa các cột bằng dummy variables và chuyển đổi thành giá trị 0 và 1
dummies = pd.get_dummies(df[columns_to_encode], prefix=columns_to_encode, drop_first=True).astype(int)

# Thêm các cột đã mã hóa vào DataFrame gốc
df = pd.concat([df, dummies], axis=1)

# Loại bỏ các cột gốc đã được mã hóa
df = df.drop(columns=columns_to_encode)

# Hiển thị DataFrame sau khi thực hiện
df

,Test Results,Gender_Male,Blood Type_A-,Blood Type_AB+,Blood Type_AB-,Blood Type_B+,Blood Type_B-,Blood Type_O+,Blood Type_O-,Medical Condition_Asthma,...,Admission Type_Emergency,Admission Type_Urgent,Medication_Ibuprofen,Medication_Lipitor,Medication_Paracetamol,Medication_Penicillin,Insurance Provider_Blue Cross,Insurance Provider_Cigna,Insurance Provider_Medicare,Insurance Provider_UnitedHealthcare
0,Inconclusive,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,Normal,1,0,0,0,0,0,1,0,1,...,1,0,0,1,0,0,0,0,0,1
2,Normal,1,0,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,1,0
3,Abnormal,1,0,0,0,0,1,0,0,1,...,0,1,0,0,0,1,0,0,1,0
4,Normal,1,0,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Abnormal,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
9996,Normal,0,0,1,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,0,0
9997,Normal,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
9998,Normal,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1


## Mã hóa các cột phụ thuộc

In [171]:
from sklearn.preprocessing import LabelEncoder

# Khởi tạo LabelEncoder object
label_encoder = LabelEncoder()

# Mã hóa chuỗi thành số nguyên và gán kết quả vào cột cũ
df['Test Results'] = label_encoder.fit_transform(df['Test Results'])

# Hiển thị dataframe với cột "furnishingstatus" đã được mã hóa
df


,Test Results,Gender_Male,Blood Type_A-,Blood Type_AB+,Blood Type_AB-,Blood Type_B+,Blood Type_B-,Blood Type_O+,Blood Type_O-,Medical Condition_Asthma,...,Admission Type_Emergency,Admission Type_Urgent,Medication_Ibuprofen,Medication_Lipitor,Medication_Paracetamol,Medication_Penicillin,Insurance Provider_Blue Cross,Insurance Provider_Cigna,Insurance Provider_Medicare,Insurance Provider_UnitedHealthcare
0,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,2,1,0,0,0,0,0,1,0,1,...,1,0,0,1,0,0,0,0,0,1
2,2,1,0,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,1,0
3,0,1,0,0,0,0,1,0,0,1,...,0,1,0,0,0,1,0,0,1,0
4,2,1,0,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
9996,2,0,0,1,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,0,0
9997,2,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
9998,2,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1


In [172]:
df['Test Results'].value_counts()

Test Results
0    3456
1    3277
2    3267
Name: count, dtype: int64

## Chia mô hình

In [173]:
feature = df.iloc[:,1:]
target = df.iloc[:,0]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=42)

## Tạo mô hình - huấn luyện - dự đoán

In [174]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train, y_train)

SVC(kernel='linear', random_state=0)

In [175]:
y_predict = classifier.predict(X_test)
print(y_predict)

[0 0 0 ... 1 0 0]


In [176]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_predict)
print(cm)
accuracy_score(y_test, y_predict)

[[481 173  52]
 [437 152  47]
 [447 161  50]]


0.3415